In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from typing import List, TypedDict, Annotated
"""
from langchain.vectorstores import FAISS
from langchain_core.graph import Graph, Node, StartNode, StopNode
from langchain.docstore.document import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
"""

# Load environment variables
load_dotenv()
api_key = os.environ["groq"]

# Initialize LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key
)


In [2]:
llm.invoke("hi").content

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [3]:
llm.invoke("where is taj mahal?").content

"The Taj Mahal is located in Agra, a city in the Indian state of Uttar Pradesh. It is situated on the southern bank of the Yamuna River, approximately 230 kilometers (143 miles) south of New Delhi, the capital city of India.\n\nThe exact address of the Taj Mahal is:\nTaj Mahal, Dharmapuri, Forest Colony, Tajganj, Agra, Uttar Pradesh 282001, India\n\nIt's one of the most famous and iconic landmarks in India, and a popular tourist destination, attracting millions of visitors each year."

In [27]:
from langchain_core.prompts import PromptTemplate
import yaml
with open('prompts.yaml', 'r') as file:
    data = yaml.safe_load(file)
    
def intro():

    """Choose a key from the following list to use the corresponding prompt template:"""
    print("*"*100)
    for i,j in enumerate(data.keys()):
        print(f"{i+1}. {j}")
    i=input("Enter the number corresponding to the key you want to use: ")
    print("selected number is ",i)
    key=list(data.keys())[int(i)-1]
    return key

def func(key):
    prompt_template = PromptTemplate(
        input_variables=["resume", "job_description"],
        template=data[key]["prompt"]
    )
    chain=prompt_template | llm
    response=chain.invoke(data[key]["inputs"]).content
    return response

In [ ]:
"""
write a function that takes input from user and uses llm to use which key 
"""